# Generate random model traffic with v2 Agents in Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 1
Get projects from foundry

In [ ]:
import sys
import os
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient

sys.path.insert(1, "./src")  # add code directory to path for imports

from foundry_utils import get_ai_foundry_projects

load_dotenv(override=True)

deployments = ["gpt-4.1-mini", "gpt-5-mini", "o3-mini"]
subscription_id = os.environ.get("AZURE_AI_FOUNDRY_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP")
account_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")

# Get all project endpoints
project_endpoints = await get_ai_foundry_projects(
    subscription_id=subscription_id,
    resource_group=resource_group,
    account_name=account_name,
)

# Create AIProjectClient and agents_utils for each project
project_clients = []
creds = DefaultAzureCredential()

for endpoint in project_endpoints:
    if endpoint:
        project_client = AIProjectClient(endpoint=endpoint, credential=creds)
        await project_client.__aenter__()
        project_clients.append(project_client)
        print(f"Created client for endpoint: {endpoint}")

print(f"\nTotal clients created: {len(project_clients)}")

## Run Tests

In [ ]:
# Send random requests to random models for each project
import sys

sys.path.insert(1, "./src")  # add code directory to path for imports
from foundry_utils import get_gateway_connections
from agents_utils import agents_utils

import pandas as pd
import random
import time
from openai import APIStatusError, APIError
from azure.core.exceptions import HttpResponseError


# Seed random with current time to ensure different values each run
random.seed(time.time())

# Configuration
min_requests_per_project = 3
max_requests_per_project = 10
prompts = [
    "What is 2 + 2?",
    "Tell me a joke",
    "What's the capital of France?",
    "Explain quantum computing in one sentence",
    "What is the meaning of life?",
    "Write a haiku about coding",
    "What color is the sky?",
    "Name three programming languages",
]

results = []

for idx, (project_client) in enumerate(project_clients):
    endpoint = project_endpoints[idx]
    num_requests = random.randint(min_requests_per_project, max_requests_per_project)
    gateway_connections = await get_gateway_connections(project_client)
    print(f"Gateway connections: {gateway_connections}")
    agents_client = agents_utils(project_client)
    connection_name = (
        gateway_connections["ai_gateway_connection_dynamic"]
        or gateway_connections["ai_gateway_connection_static"]
    )
    print(f"Using connection: {connection_name}")
    if not connection_name:
        print(f"No gateway connection found for project {endpoint}, skipping...")
        continue

    print(f"\n{'='*60}")
    print(f"Project {idx + 1}: {endpoint}")
    print(f"Sending {num_requests} random requests")
    print("=" * 60)

    openai_client = project_client.get_openai_client()

    for i in range(num_requests):
        # Pick random model and prompt
        model = random.choice(deployments)
        prompt = random.choice(prompts)
        request_id = None
        result = None
        error_msg = None
        output = None

        try:
            agent = await agents_client.create_agent(
                name="temp-agent",
                model_gateway_connection=connection_name,
                deployment_name=model,
                delete_before_create=False,
            )

            # Create conversation
            conversation = await openai_client.conversations.create(
                items=[
                    {
                        "type": "message",
                        "role": "user",
                        "content": prompt,
                    }
                ],
            )

            # Get response
            response = await openai_client.responses.create(
                conversation=conversation.id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input="",
            )

            request_id = response._request_id
            result = "SUCCESS"
            output = response.output_text[:100] if response.output_text else "No output"

            print(
                f"  [{i+1}/{num_requests}] Model: {model} | SUCCESS | {output[:50]}..."
            )

        except (APIStatusError, APIError, HttpResponseError) as e:
            request_id = (
                e.response.headers.get("x-request-id", "N/A")
                if hasattr(e, "response") and e.response
                else "N/A"
            )
            result = "ERROR"
            error_msg = str(e)
            output = error_msg
            print(f"  [{i+1}/{num_requests}] Model: {model} | ERROR | {error_msg}...")

        except Exception as e:
            result = "ERROR"
            error_msg = str(e)
            output = error_msg
            print(f"  [{i+1}/{num_requests}] Model: {model} | ERROR | {error_msg}...")

        results.append(
            {
                "Project": endpoint.split("/")[-1] if endpoint else f"Project_{idx}",
                "Model": model,
                "Request #": i + 1,
                "Request ID": request_id or "N/A",
                "Prompt": prompt[:30] + "..." if len(prompt) > 30 else prompt,
                "Result": result,
                "Output": output[:50] if output else "N/A",
            }
        )

# Create DataFrame and display results
df = pd.DataFrame(results)
print("\n\n" + "=" * 80)
print("=== RESULTS TABLE ===")
print("=" * 80)
print(df.to_string(index=False))

# Save results to CSV
df.to_csv("gateway_random_traffic_results.csv", index=False)
print("\nResults saved to gateway_random_traffic_results.csv")

# Summary statistics
print("\n\n=== SUMMARY BY PROJECT AND MODEL ===")
summary = df.groupby(["Project", "Model", "Result"]).size().unstack(fill_value=0)
print(summary)

print("\n\n=== SUMMARY BY RESULT ===")
print(df["Result"].value_counts())